In [68]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [69]:
class CancerDataset(Dataset):

    def __init__(self, file_name, train=False):
        
        data = pd.read_csv(file_name).iloc[:, 1:-1]
        
        if train:
            x = data.iloc[:, 1:].values[:400]
            y = pd.get_dummies(data.iloc[:, 0]).values[:400]
        else:
            x = data.iloc[:, 1:].values[400:]
            y = pd.get_dummies(data.iloc[:, 0]).values[400:]
        
        
        sc = StandardScaler()
        x_train = sc.fit_transform(x)
        y_train = y
        
        self.X_train = torch.tensor(x_train, dtype=torch.float32)
        self.Y_train = torch.tensor(y_train, dtype=torch.float32)
        
    def __len__(self):
        return len(self.Y_train)

    def __getitem__(self, idx):
        return self.X_train[idx], self.Y_train[idx]
    
    
    
training_set = CancerDataset('data.csv', train=True)
testing_set = CancerDataset('data.csv')

training_loader = DataLoader(training_set, batch_size=4, shuffle=True)
testing_loader = DataLoader(testing_set, batch_size=4, shuffle=False)

In [70]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.linear1 = nn.Linear(30, 2)
        self.activation = nn.ReLU()
        self.linear2 = nn.Linear(2, 2)
        self.softmax = nn.Softmax(dim=1)

    
    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.softmax(x)
        
        return x

In [71]:
import torch.optim as optim

net = Net()

optimizer = optim.SGD(net.parameters(), lr=0.01)
criterion = nn.MSELoss()


for epoch in range(2):
    for i, data in enumerate(training_loader, 0):
        inputs, labels = data
        
        optimizer.zero_grad()
        
        output = net(inputs)
        # print(output, labels)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step() 

In [72]:
PATH = './cancer_net.pth'
torch.save(net.state_dict(), PATH)

In [73]:
correct, total = 0, 0
with torch.no_grad():
    for data in testing_loader:
        inputs, labels = data
        outputs = net(inputs)
        
        _, predicted = torch.max(outputs, 1)
        _, y = torch.max(labels, 1)
        
        total += labels.size(0)
        correct += (predicted == y).sum().item()
        
print(f'Accuracy of nn: {correct / total}')

Accuracy of nn: 0.9349112426035503
